In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

Custom functions

In [134]:
#1 Remove stop words from text
def remove_stops(text, filler=''):
    return filler.join([word.title() for word in text.lower().split() if word not in stopwords.words('english')])

#2 Plot function for value counts
def value_count_plot(df, columns, size=[10, 10]):
    ncols = 3
    idx = 1

    if len(columns) % ncols != 0:
        nrows = len(columns) // ncols + 1
    else:
        nrows = len(columns) // ncols

    plt.figure(figsize=size, layout='tight')
    for col in columns:
        if idx == 3:
            ax.set_title('Value counts for object types')
        ax = plt.subplot(nrows, ncols, idx)
        df[col].value_counts(normalize= True).plot.barh(ax)
        idx += 1

    plt.show()

## Load Data

In [118]:
ls = pd.read_csv("Leads.csv")
ls.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [119]:
ls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

Prospect ID is an object type unique key for all records, but so is Lead Number which is an integer type. Hence Prospect ID is dropped and Lead Number can be used as unique key of each record

In [120]:
ls.drop(columns=['Prospect ID'], inplace= True)
ls.head(1)

,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified


Shortening column names by removing stopwords and other chosen words

In [121]:
ls.columns = [remove_stops(text, '_') for text in ls.columns]

Percent null values

In [122]:
nulls = ls.isnull().sum()
condition = ls.isnull().sum() > 0

df_nulls = pd.concat([ls[nulls[condition].index].dtypes, (nulls[condition] / len(ls) * 100).round(2)], axis=1).reset_index()
df_nulls.columns = ['Column_name', 'dtype', 'percent_nulls']
df_nulls.sort_values(by= 'percent_nulls', ascending= False)

,Column_name,dtype,percent_nulls
10,Lead_Quality,object,51.59
16,Asymmetrique_Profile_Score,float64,45.65
13,Asymmetrique_Activity_Index,object,45.65
15,Asymmetrique_Activity_Score,float64,45.65
14,Asymmetrique_Profile_Index,object,45.65
9,Tags,object,36.29
8,Matters_Choosing_Course,object,29.32
11,Lead_Profile,object,29.32
7,Current_Occupation,object,29.11
4,Country,object,26.63


Data type segregation

In [123]:
obj_cols = ls.columns[ls.dtypes == 'O']
num_cols = ls.columns[ls.dtypes != 'O']
num_cols = num_cols.drop('Lead_Number')

print('Number of object types', len(obj_cols))
print('Number of number types', len(num_cols))

Number of object types 29
Number of number types 6


## Missing value treatment